In [83]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
from collections import Counter
import numpy as np

news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1') # 뉴스 raw data
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1') # stock_list data
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1') # 대표단어 csv

# 테스트 용도로 데이터 1000개만 가지고
news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower() #소문자화
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
self_stop_words = {'bloomberg'}
news_raw['summary'].replace(self_stop_words, '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

news_raw['company_name'] = 0
news_raw['keyword'] = 0 # keybert로 뽑아낸 키워드 컬럼
news_raw['keyword2'] = 0

# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

## 회사 이름 추출 => 추출 완료 ( company_word.csv 에 저장 )

In [ ]:
# 회사이름 추출 데이터 저장용도
company_word = pd.DataFrame(columns=['news_id', 'company_word','pk'])

# 대표단어 csv파일에서 대표단어가 포함되면 맵핑
i = 0

for token_num, token in enumerate(tokenize_list):  # news_raw data 토큰화
    for rep_num, rep in enumerate(rep_list):  # 대표단어 csv파일에서 대표단어에 해당
        i = i + 1
        if len(rep) == 1:
            if rep[0] in token:
                company_word.loc[i] = [news_raw['news_id'][token_num], rep[0], rep_stock['pk'][rep_num]]

        elif len(rep) == 2:  # 대표단어가 2개로 된 단어일 때
            if rep[0] in token:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = token.index(rep[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = token[search]  # org의 (+1을 한) 다음 단어에 해당
                    if rep[1] == search_found:
                        company_word.loc[i] = [news_raw['news_id'][token_num], rep[0] + " " + rep[1],
                                                 rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

        elif len(rep) == 3:
            if rep[0] in token:
                try:
                    search_found = token[token.index(rep[0]) + 1]
                    if rep[1] == search_found:
                        two_found = token[token.index(rep[0]) + 2]
                        if rep[2] == two_found:
                            company_word.loc[i] = [news_raw['news_id'][token_num],
                                                     rep[0] + " " + rep[1] + " " + rep[2], rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

company_word = pd.merge(company_word, nasdaq_stock, how='left', left_on='pk', right_on='pk')
#company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')

## 뉴스 원문에서 키워드 추출 ( Keybert 사용 )=> 추출 완료 ( keyword.csv 에 저장 )

In [ ]:
# Keybert로 키워드 추출 ( 키워드 1단어들, 2단어들 )
summaries = news_raw['summary']
# 뉴스 원문 키워드 10개 추출 저장 컬럼
news_raw['news_ten_keyword'] = 0

for sum_num, summary in enumerate(summaries):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(summary,top_n=5)
    news_raw['keyword'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1)) # 뉴스 원문 키워드 한 단어들 다섯개 추출
    news_raw['keyword2'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 2)) # 뉴스 원문 키워드 두 단어들 다섯개 추출
    news_raw['ten_keyword'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1),top_n= 10) # 뉴스 원문 키워드 10개 추출
#news_raw.to_csv('C:/projectnasdaq/project2/keyword.csv')

## 종목 정보 수집 ( yfinance 패키지 사용 ) => 수집완료

In [ ]:
# 종목코드 리스트
stocks = nasdaq_stock['symbol']
# 종목코드에 대한 정보 수집용 데이터프레임 생성
stock_info = pd.DataFrame(columns=['symbol', 'info'])
i = 0

for stock_num, stock in enumerate(stocks):
    tickers = yf.Ticker(stock)
    ticker = tickers.info
    i = i + 1
    #print(stock_num,stock, "===>>",ticker)
    stock_info.loc[i] = [stock, ticker]
    time.sleep(random.uniform(3, 4))

#stock_info.to_csv('C:/projectnasdaq/project2/stock_info.csv')

## 추출된 회사 이름이랑 ( 전처리 한 ) 키워드 연결 ( company_and_keyword.csv 파일에 저장 )

In [87]:
company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')
keyword = pd.read_csv('C:/projectnasdaq/project2/keyword.csv')

# 키워드 전처리
keyword['keyword'] = keyword['keyword'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
keyword['keyword'] = keyword['keyword'].apply(lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
keyword['keyword'] = keyword['keyword'].str.split(" ")

keyword['news_keyword_ten'] = keyword['news_keyword_ten'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
keyword['news_keyword_ten'] = keyword['news_keyword_ten'].apply(lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
keyword['news_keyword_ten'] = keyword['news_keyword_ten'].str.split(" ")

keyword['keyword2'] = keyword['keyword2'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
keyword['keyword2'] = keyword['keyword2'].str.split("  ")


# 공백으로 토큰화되거나 비어있는 내용으로 토큰화 되어있으면 지우기
keyword['keyword2'] = keyword['keyword2'].apply(lambda x: [i for i in x if i != "" and i != " "])

company_word[['keyword', 'keyword2','news_keyword_ten']] = 0

keyword_list1 = keyword['keyword']
keyword_list2 = keyword['keyword2']

keyword_id = keyword['news_id']
company_id = company_word['news_id']

for i, j in enumerate(company_id):
    for k, l in enumerate(keyword_id):
        if j == l:
            company_word['keyword'][i] = keyword['keyword'][k]
            company_word['keyword2'][i] = keyword['keyword2'][k]

#company_mapping.to_csv('C:/projectnasdaq/project2/company_and_keyword.csv')

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

## yahoo_dataset 파일 수정 ( yahoo_dataset_mapping.csv 에 저장 )

In [88]:
# yahoo_dataset 파일 수정 ( -> symbol name 수정 )
yahoo_dataset = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset.csv')
# yahoo_dataset에서 필요없는 컬럼들 지우고 csv 파일에 있는 종목코드에 종목명 데이터들 붙임

yahoo_dataset.insert(1,'new_symbol','0')
yahoo_dataset.insert(2,'name','0')
yahoo_dataset.insert(3,'company_word','0')

yahoo_dataset_symbol = yahoo_dataset['symbol']
rep_stock_symbol = rep_stock['symbol']

for i, j in enumerate(yahoo_dataset_symbol):
    for k, l in enumerate(rep_stock_symbol):
        if j == l:
            yahoo_dataset['new_symbol'][i] = rep_stock['symbol'][k]
            yahoo_dataset['name'][i] = rep_stock['name'][k]
            yahoo_dataset['company_word'][i] = rep_stock['name_a'][k]
            break

# yahoo_dataset 컬럼 정리
yahoo_dataset.drop(columns=["quoteType", "currency", 'regularMarketPrice', 'regularMarketChange', 'regularMarketChangePercent',
             'regularMarketVolume', 'averageDailyVolume3Month', 'marketCap', 'trailingPE', 'fiftyTwoWeekLow',
             'fiftyTwoWeekHigh', 'regularMarketOpen', 'priceHint', 'underlyingSymbol'], inplace=True)
#yahoo_dataset.to_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

## 종목명이 추출된 뉴스 키워드들끼리 모으기 ( 딕셔너리 형태로 )

In [119]:
# company단어들 하나씩만 list에 담기 -> 단어 이름이 같은 종목명들의 키워드 모으기 위해서
company_and_keyword = pd.read_csv('C:/projectnasdaq/project2/company_and_keyword.csv')
company_and_keyword.drop(columns=["Unnamed: 0", 'Unnamed: 0.1'], inplace=True)
company_word = company_and_keyword['company_word']

company_word_same_list = []
for company in company_word:
    if company not in company_word_same_list:
        company_word_same_list.append(company)

company_word_sames = pd.DataFrame(company_word_same_list, columns=['company_word_same'])
company_word_same = company_word_sames['company_word_same']

# 키워드랑 company_name이랑 데이터프레임에서 맵핑 시키기위한 새로운 dataframe 생성
company_name_keyword_organize = pd.DataFrame(columns=['company', 'keyword', 'keyword2','key_list'])

# 딕셔너리를 { news_id : keyword } 형태로 만들기 위해서 index를 news_id로 설정
company_and_keyword = company_and_keyword.set_index(keys='news_id', drop=False, inplace=False)

# company_name에 대한 키워드들 딕셔너리 형태로 모으기
for i, j in enumerate(company_word_same):
    company_name_keyword_organize.loc[i] = [j, company_and_keyword[company_and_keyword['company_word'] == j]['keyword'].to_dict(), company_and_keyword[company_and_keyword['company_word'] == j]['keyword2'].to_dict(),company_and_keyword[company_and_keyword['company_word'] == j]['keyword'].tolist()] # 리스트 형태로 키워드만 뽑아오기 위해서

## news_raw_dataset 만들기

In [120]:
# company_name_keyword_organize.csv에 column 삽입
company_name_keyword_organize.insert(0, 'symbol', '0')
company_name_keyword_organize.insert(1, 'name', '0')
company_name_keyword_organize.insert(2, 'shortName', '0')
company_name_keyword_organize.insert(3, 'longName', '0')
company_name_keyword_organize.insert(5, 'company_info', '0')

In [121]:
yahoo_dataset_mapping = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')
company_word_csv = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')

# company_name_keyword_organize.csv에 symbol, name 데이터 삽입
company_word = company_word_csv['company_word']
company = company_name_keyword_organize['company']

for i,j in enumerate(company_word):
    for k, l in enumerate(company):
        if j==l:
            company_name_keyword_organize['symbol'][k] = company_word_csv['symbol'][i]
            company_name_keyword_organize['name'][k] = company_word_csv['name'][i]
            break

# company__name.csv에 yahoo finance 정보 수집 내용 삽입
yahoo_dataset_mapping_symbol = yahoo_dataset_mapping['new_symbol']
company_name_keyword_organize_symbol = company_name_keyword_organize['symbol']

for i,j in enumerate(yahoo_dataset_mapping_symbol):
    for k,l in enumerate(company_name_keyword_organize_symbol):
        if j==l:
            company_name_keyword_organize['shortName'][k] = yahoo_dataset_mapping['shortName'][i]
            company_name_keyword_organize['longName'][k] = yahoo_dataset_mapping['longName'][i]
            company_name_keyword_organize['company_info'][k] = yahoo_dataset_mapping['company_info'][i]
            break

# 대표 키워드 추출하기
# 키워드 빈도수 순으로 정리해서 5개까지만 뽑아오기
company_name_keyword_organize['frequency_4'] = 0
company_name_keyword_organize['frequency_5'] = 0
company_name_keyword_organize

key_list_ = company_name_keyword_organize['key_list']
for i, j in enumerate(key_list_):
        company_name_keyword_organize['key_list'][i] = ''.join(j)

company_name_keyword_organize['key_list'].replace(['\]','\[','\''],'',regex=True, inplace=True)
company_name_keyword_organize['key_list'].replace(',',' ',regex=True, inplace=True)
company_name_keyword_organize['key_list'] = company_name_keyword_organize['key_list'].str.split(" ")
company_name_keyword_organize['key_list'] = company_name_keyword_organize['key_list'].apply(lambda x: [i for i in x if i != "" and i != " "])


frequency_word = company_name_keyword_organize['key_list']
for i, j in enumerate(frequency_word):
    # 대표 키워드 빈도순으로 5개 추출
    company_name_keyword_organize['frequency_4'][i] = Counter(j).most_common(4)
    company_name_keyword_organize['frequency_5'][i] = Counter(j).most_common(5)
# 다섯개 뽑아온 키워드 전처리
company_name_keyword_organize['frequency_4'] = company_name_keyword_organize['frequency_4'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
company_name_keyword_organize['frequency_4'] = company_name_keyword_organize['frequency_4'].str.split(" ")
company_name_keyword_organize['frequency_4'] = company_name_keyword_organize['frequency_4'].apply(lambda x: [i for i in x if i != "" and i != " "])

company_name_keyword_organize['frequency_5'] = company_name_keyword_organize['frequency_5'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
company_name_keyword_organize['frequency_5'] = company_name_keyword_organize['frequency_5'].str.split(" ")
company_name_keyword_organize['frequency_5'] = company_name_keyword_organize['frequency_5'].apply(lambda x: [i for i in x if i != "" and i != " "])

# 완성된 news_raw dataset
#company__name.to_csv('C:/projectnasdaq/project2/news_raw_dataset.csv')

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, nam

In [122]:
company_name_keyword_organize

,symbol,name,shortName,longName,company,company_info,keyword,keyword2,key_list,frequency_4,frequency_5
0,KO,Coca-Cola Company (The),Coca-Cola Company (The),The Coca-Cola Company,coca cola,"{""address1"": ""One Coca-Cola Plaza"", ""city"": ""A...","{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf...","[nestle, starbucks, brands, bottled, marketpos...","[cola, nestle, starbucks, brands]","[cola, nestle, starbucks, brands, bottled]"
1,UL,Unilever PLC,Unilever PLC,Unilever PLC,unilever,"{""address1"": ""Unilever House"", ""address2"": ""10...","{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf...","[nestle, starbucks, brands, bottled, marketpal...","[nestle, starbucks, brands, bottled]","[nestle, starbucks, brands, bottled, marketpalm]"
2,DEO,Diageo plc,Diageo plc,Diageo plc,diageo,"{""address1"": ""Lakeside Drive"", ""address2"": ""Pa...","{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf...","[nestle, starbucks, brands, bottled, marketdia...","[nestle, starbucks, brands, bottled]","[nestle, starbucks, brands, bottled, marketdia..."
3,BUD,Anheuser-Busch Inbev SA Sponsored ADR (Belgium),Anheuser-Busch Inbev SA Sponsor,Anheuser-Busch InBev SA/NV,anheuser busch,"{""address1"": ""Brouwerijplein 1"", ""city"": ""Leuv...","{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf...","[nestle, starbucks, brands, bottled, marketinv...","[nestle, starbucks, brands, bottled]","[nestle, starbucks, brands, bottled, marketinv..."
4,SBUX,Starbucks Corporation,Starbucks Corporation,Starbucks Corporation,starbucks,"{""address1"": ""2401 Utah Avenue South"", ""city"":...","{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf...","[nestle, starbucks, brands, bottled, marketdis...","[starbucks, shenzhen, nestle, brands]","[starbucks, shenzhen, nestle, brands, bottled]"
...,...,...,...,...,...,...,...,...,...,...,...
425,DAL,Delta Air Lines Inc,"Delta Air Lines, Inc.","Delta Air Lines, Inc.",delta air,"{""address1"": ""PO Box 20706"", ""city"": ""Atlanta""...","{'443844': '['pandemic', 'airline', 'airlines'...","{'443844': '['airline inevitable', 'pandemic c...","[pandemic, airline, airlines, coronavirus, fli...","[pandemic, airline, airlines, coronavirus]","[pandemic, airline, airlines, coronavirus, fli..."
426,GDRX,GoodRx Holdings Inc,"GoodRx Holdings, Inc.","GoodRx Holdings, Inc.",goodrx holdings,"{""address1"": ""2701 Olympic Boulevard"", ""addres...","{'443702': '['recession', 'stocks', 'stocksthe...","{'443702': '['rampant stock', ' stocks slumped...","[recession, stocks, stocksthe, markets, stock]","[recession, stocks, stocksthe, markets]","[recession, stocks, stocksthe, markets, stock]"
427,SPI,SPI Energy Co,"SPI Energy Co., Ltd.","SPI Energy Co., Ltd.",spi energy,"{""address1"": ""4803 Urbani Avenue"", ""address2"":...","{'443710': '['tesla', 'ev', 'spi', 'stocks', '...","{'443710': '['energy stocks', ' spi stock', 'i...","[tesla, ev, spi, stocks, stock]","[tesla, ev, spi, stocks]","[tesla, ev, spi, stocks, stock]"
428,PBR,Petroleo Brasileiro S,Petroleo Brasileiro S.A.- Petro,Petróleo Brasileiro S.A. - Petrobras,petroleo brasileiro,"{""address1"": ""Avenida RepUblica do Chile, 65"",...","{'443728': '['vitol', 'indictment', 'prosecuto...","{'443728': '['bribes vitol', 'vitol firm', 'au...","[vitol, indictment, prosecutors, indicted, pro...","[vitol, indictment, prosecutors, indicted]","[vitol, indictment, prosecutors, indicted, pro..."


## 맵핑 (1) 뉴스 원문에 종목명의 대표 키워드가 포함될 때

In [193]:
# 뉴스 원문에 대표 키워드가 포함될 경우 news_id와 symbol 맵핑
frequency_4 = company_name_keyword_organize['frequency_4'] # 키워드 빈도수로 정렬 후 4개만 담아온 리스트
news_stock_mapping = pd.DataFrame(columns=['news_id', 'symbol','name','summary','frequency_4'])

a = 0
for i, j in enumerate(tokenize_list):
    for k,l in enumerate(frequency_4):
        a = a + 1
        #print(news_raw['news_id'][i])
        together = set(j)&set(l)
        # 뉴스 원문에 대표 키워드가 4개 이상 교집합으로 있을 경우
        if len(together)==4:
            news_stock_mapping.loc[a] = [news_raw['news_id'][i], company_name_keyword_organize['symbol'][k], company_name_keyword_organize['name'][k],news_raw['summary'][i], company_name_keyword_organize['frequency_4'][k]]

In [194]:
print('총 맵핑 수 :',len(news_stock_mapping))

총 맵핑 수 : 1290


## 맵핑(2) 뉴스 원문 키워드(10개)랑 종목명의 대표 키워드가 일치할 때

In [195]:
news_stock_mapping_2 = pd.DataFrame(columns=['news_id', 'symbol','name','summary','frequency_5','news_keyword_ten'])
news_raw_keyword_ten = keyword['news_keyword_ten']
frequency_5 = company_name_keyword_organize['frequency_5']
# frequency => 종목명의 대표키워드
a = 0
for i,j in enumerate(news_raw_keyword_ten):
    for k,l in enumerate(frequency_5):
        a = a + 1
        together = set(j)&set(l)
        if len(together)>=4:
            news_stock_mapping_2.loc[a] = [keyword['news_id'][i], company_name_keyword_organize['symbol'][k], company_name_keyword_organize['name'][k],keyword['summary'][i], company_name_keyword_organize['frequency_5'][k],keyword['news_keyword_ten'][i]]

In [196]:
print('총 맵핑 수 :',len(news_stock_mapping_2))

총 맵핑 수 : 1069


In [197]:
news_stock_mapping = news_stock_mapping.drop(columns=['frequency_4'])
news_stock_mapping_2 = news_stock_mapping_2.drop(columns=['frequency_5', 'news_keyword_ten'])
news_stock_mapping_final = pd.concat([news_stock_mapping,news_stock_mapping_2], ignore_index=True)
overlap = news_stock_mapping_final.drop_duplicates(['news_id','symbol'])

In [198]:
overlap

,news_id,symbol,name,summary
0,345585,KO,Coca-Cola Company (The),nestle sa sailed past ailing consumer goo...
1,345585,UL,Unilever PLC,nestle sa sailed past ailing consumer goo...
2,345585,DEO,Diageo plc,nestle sa sailed past ailing consumer goo...
3,345585,BUD,Anheuser-Busch Inbev SA Sponsored ADR (Belgium),nestle sa sailed past ailing consumer goo...
4,345586,E,ENI S,total se made a surprise profit after ve...
...,...,...,...,...
2309,439567,NDAQ,Nasdaq Inc,stocks pared losses as a rebound in some ...
2318,441705,GOOG,Alphabet Inc,south africa s main stock benchmark trade...
2322,441738,BSAC,Banco Santander - Chile ADS,italy has asked unicredit spa executives ...
2328,441739,ALLT,Allot Ltd,the u s death toll from the novel corona...
